<a href="https://colab.research.google.com/github/LuziaManuel22/Machine_Learning_Winter_2024/blob/main/Glaucoma_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importando as bibliotecas necessárias
import pandas as pd

# 1. Carregando os dados
# Carregar o arquivo CSV com os caminhos das imagens e seus rótulos
csv_file = "/content/JustRAIGS_Train_labels.csv"  # Substitua pelo caminho do seu arquivo CSV
data = pd.read_csv(csv_file, delimiter=';')  # Especificando o delimitador como ponto e vírgula

# Encontrar e mostrar os valores ausentes
missing_values = data.isnull().sum()
missing_percentage = (missing_values / len(data)) * 100
missing_data = pd.concat([missing_values, missing_percentage], axis=1)
missing_data.columns = [' Missing Values', 'Percentage Missing']
print("Missing values and percentages per column::")
print("\n")  # Adiciona três linhas em branco
print(missing_data)
print("\n" * 3)  # Adiciona três linhas em branco

# Configurar o Pandas para exibir mais colunas e expandir o espaço entre elas
pd.set_option('display.max_columns', None)  # Exibir todas as colunas
pd.set_option('display.max_colwidth', None)  # Remover limite de largura das colunas
pd.set_option('display.expand_frame_repr', False)  # Exibir DataFrame em uma única linha
pd.set_option('display.colheader_justify', 'center')  # Centralizar os títulos das colunas

# Exibir as primeiras 21 linhas do arquivo CSV para entender a estrutura dos dados
print("First 21 lines of the CSV file::")
print("\n")  # Adiciona três linhas em branco
print(data.head(21))


Missing values and percentages per column::


                Missing Values  Percentage Missing
Eye ID                       0            0.000000
Final Label                  0            0.000000
Fellow Eye ID             3783            8.186540
Age                          9            0.019476
Label G1                   122            0.264012
Label G2                   615            1.330881
Label G3                 42408           91.772344
G1 ANRS                    122            0.264012
G1 ANRI                    122            0.264012
G1 RNFLDS                  122            0.264012
G1 RNFLDI                  122            0.264012
G1 BCLVS                   122            0.264012
G1 BCLVI                   122            0.264012
G1 NVT                     122            0.264012
G1 DH                      122            0.264012
G1 LD                      122            0.264012
G1 LC                      122            0.264012
G2 ANRS                    615      

In [ ]:
# 2. Pré-processamento dos dados
# Carregar e redimensionar as imagens, convertendo-as em arrays numpy
images = []
labels_referable_glaucoma = []
labels_additional_features = []

for index, row in data.iterrows():
    image = cv2.imread(row['image_path'])  # 'image_path' é o nome da coluna que contém os caminhos das imagens
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Converter de BGR para RGB
    image = cv2.resize(image, (224, 224))  # Redimensionar para um tamanho padrão
    images.append(image)
    labels_referable_glaucoma.append(row['referable_glaucoma'])  # Rótulos para classificação binária
    labels_additional_features.append(row.drop(['image_path', 'referable_glaucoma']).values)  # Rótulos para classificação multirrótulo

images = np.array(images)
labels_referable_glaucoma = np.array(labels_referable_glaucoma)
labels_additional_features = np.array(labels_additional_features)

In [ ]:
# 3. Divisão dos dados em conjuntos de treinamento, validação e teste
X_train, X_test, y_train_referable, y_test_referable, y_train_features, y_test_features = train_test_split(images, labels_referable_glaucoma, labels_additional_features, test_size=0.2, random_state=42)
X_train, X_val, y_train_referable, y_val_referable, y_train_features, y_val_features = train_test_split(X_train, y_train_referable, y_train_features, test_size=0.2, random_state=42)



In [ ]:
# 4. Criando o modelo
# Arquitetura baseada em uma CNN simples para classificação binária referable glaucoma
model_referable_glaucoma = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compilando o modelo
model_referable_glaucoma.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
# 5. Treinamento do modelo
history_referable_glaucoma = model_referable_glaucoma.fit(X_train, y_train_referable, epochs=10, batch_size=32, validation_data=(X_val, y_val_referable))

# Plotando a curva de aprendizado
plt.plot(history_referable_glaucoma.history['accuracy'], label='accuracy')
plt.plot(history_referable_glaucoma.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

# Avaliando o modelo
loss, accuracy = model_referable_glaucoma.evaluate(X_test, y_test_referable)
print("Test Accuracy (Referable Glaucoma):", accuracy)

In [ ]:
# 6. Criando e treinando o modelo para classificação multirrótulo das características adicionais
# Codificando os rótulos multirrótulo usando MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_train_features_encoded = mlb.fit_transform(y_train_features)
y_val_features_encoded = mlb.transform(y_val_features)
y_test_features_encoded = mlb.transform(y_test_features)

# Arquitetura do modelo para classificação multirrótulo
model_additional_features = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(mlb.classes_), activation='sigmoid')  # Usando sigmoid para múltiplas saídas
])

# Compilando o modelo
model_additional_features.compile(optimizer='adam',
                                  loss='binary_crossentropy',
                                  metrics=['accuracy'])

# Treinando o modelo
history_additional_features = model_additional_features.fit(X_train, y_train_features_encoded, epochs=10, batch_size=32, validation_data=(X_val, y_val_features_encoded))

# Plotando a curva de aprendizado
plt.plot(history_additional_features.history['accuracy'], label='accuracy')
plt.plot(history_additional_features.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

# Avaliando o modelo
loss, accuracy = model_additional_features.evaluate(X_test, y_test_features_encoded)
print("Test Accuracy (Additional Features):", accuracy)

In [ ]:
# 7. Ajustes e otimização (opcional)
# Se necessário, ajuste a arquitetura do modelo, hiperparâmetros e utilize técnicas de regularização para melhorar o desempenho

# Ajustando o modelo referable glaucoma
model_referable_glaucoma_adjusted = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compilando o modelo ajustado
model_referable_glaucoma_adjusted.compile(optimizer='adam',
                                           loss='binary_crossentropy',
                                           metrics=['accuracy'])

# Treinando o modelo ajustado
history_referable_glaucoma_adjusted = model_referable_glaucoma_adjusted.fit(X_train, y_train_referable, epochs=15, batch_size=32, validation_data=(X_val, y_val_referable))

# Avaliando o modelo ajustado
loss, accuracy = model_referable_glaucoma_adjusted.evaluate(X_test, y_test_referable)
print("Test Accuracy (Referable Glaucoma - Adjusted):", accuracy)

# Ajustando o modelo de características adicionais
model_additional_features_adjusted = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(mlb.classes_), activation='sigmoid')
])

# Compilando o modelo ajustado
model_additional_features_adjusted.compile(optimizer='adam',
                                           loss='binary_crossentropy',
                                           metrics=['accuracy'])

# Treinando o modelo ajustado
history_additional_features_adjusted = model_additional_features_adjusted.fit(X_train, y_train_features_encoded, epochs=15, batch_size=32, validation_data=(X_val, y_val_features_encoded))

# Avaliando o modelo ajustado
loss, accuracy = model_additional_features_adjusted.evaluate(X_test, y_test_features_encoded)
print("Test Accuracy (Additional Features - Adjusted):", accuracy)


In [ ]:
# 8. Implantação (opcional)
# Após treinar e ajustar os modelos, você pode implantá-los em um ambiente de produção para classificar novas imagens de fundo de olho
# Salvar os modelos treinados
model_referable_glaucoma.save("model_referable_glaucoma.h5")
model_additional_features.save("model_additional_features.h5")